In [1]:
# !pip uninstall sklearn


In [2]:
# !pip install scikit-learn==1.0

In [3]:
# !pip uninstall imbalanced-learn

^C


In [10]:
# !pip install imbalanced-learn==0.9.0

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
import os
import numpy as np
import pandas as pd
from collections import Counter

In [12]:
from sklearn import metrics, tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.ensemble import EasyEnsembleClassifier,  BalancedRandomForestClassifier
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [13]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [16]:
# Load the data
file_path = os.path.join("data",'LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [17]:
df["issue_d"] = pd.to_datetime(df["issue_d"])
df["issue_d"] = pd.to_numeric(df["issue_d"])
df["next_pymnt_d"] = pd.to_datetime(df["next_pymnt_d"])
df["next_pymnt_d"] = pd.to_numeric(df["next_pymnt_d"])
df["hardship_flag"] = df["hardship_flag"] != 'N'  # N = False
df["debt_settlement_flag"] = df["debt_settlement_flag"] != "N" # N = False
df["pymnt_plan"] = df["pymnt_plan"] != 'n'  # n = False
df["loan_status"] = df["loan_status"] != "low_risk" # low_risk = FALSE ; high_risk = TRUE
pd.get_dummies(df, columns=["home_ownership","verification_status", "application_type", "initial_list_status"])
cleaned_df = df.drop(columns=["home_ownership","verification_status", "application_type", "initial_list_status"])
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 82 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   annual_inc                  68817 non-null  float64
 4   issue_d                     68817 non-null  int64  
 5   loan_status                 68817 non-null  bool   
 6   pymnt_plan                  68817 non-null  bool   
 7   dti                         68817 non-null  float64
 8   delinq_2yrs                 68817 non-null  float64
 9   inq_last_6mths              68817 non-null  float64
 10  open_acc                    68817 non-null  float64
 11  pub_rec                     68817 non-null  float64
 12  revol_bal                   68817 non-null  float64
 13  total_acc                   688

# Split the Data into Training and Testing

In [18]:
# Create our features
X = StandardScaler().fit_transform(cleaned_df.drop(columns="loan_status"))
X = pd.DataFrame(X, columns=[x for x in cleaned_df.columns if x != 'loan_status'])
# Create our target
y = cleaned_df["loan_status"]

In [19]:
X.head()

,loan_amnt,int_rate,installment,annual_inc,issue_d,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,-0.601093,0.917984,-0.365558,-0.192195,1.670601,0.0,0.270401,-0.303143,-0.656492,-0.761659,...,-1.123855,2.062779,-0.374103,0.0,-0.748655,-0.403213,-1.035041,0.122936,0.0,0.0
1,0.809787,1.501829,1.556747,0.145236,1.670601,0.0,-0.076645,-0.303143,-0.656492,0.732656,...,-0.463303,0.576065,2.595637,0.0,0.318420,-0.012153,0.427908,-0.128054,0.0,0.0
2,0.323277,1.501829,0.170892,-0.278716,1.670601,0.0,0.122869,-0.303143,-0.656492,-0.761659,...,-3.405763,0.576065,-0.374103,0.0,-0.774811,-0.272784,-0.829780,-0.246838,0.0,0.0
3,-0.649744,0.753843,-0.441237,0.032759,1.670601,0.0,0.478331,-0.303143,0.662568,-0.429589,...,0.593581,0.576065,2.595637,0.0,-0.573252,0.129742,-0.374475,0.407857,0.0,0.0
4,0.517881,0.408938,0.137947,-0.313324,1.670601,0.0,-0.149420,-0.303143,0.662568,0.234551,...,0.593581,-0.910648,-0.374103,0.0,0.050396,-0.617197,-0.079646,-0.701016,0.0,0.0


In [20]:
# Check the balance of our target values
y.value_counts()

False    68470
True       347
Name: loan_status, dtype: int64

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=9, stratify=y) 
model = tree.DecisionTreeClassifier()

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [22]:
# Resample the training data with the BalancedRandomForestClassifier
br_forest_model = BalancedRandomForestClassifier(n_estimators=100, random_state=9)
br_forest_model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=9)

In [23]:
# Calculated the balanced accuracy score
br_forest_predict = br_forest_model.predict(X_test)
print("Accuracy Score: ", metrics.accuracy_score(y_test, br_forest_predict))

Accuracy Score:  0.9149665794827085


In [24]:
# Display the confusion matrix
print(metrics.confusion_matrix(y_test, br_forest_predict))

[[15687  1431]
 [   32    55]]


In [25]:
# Print the imbalanced classification report
print("False = Low Risk","\nTrue = High Risk")
print(classification_report_imbalanced(y_test, br_forest_predict))

False = Low Risk 
True = High Risk
                   pre       rec       spe        f1       geo       iba       sup

      False       1.00      0.92      0.63      0.96      0.76      0.60     17118
       True       0.04      0.63      0.92      0.07      0.76      0.56        87

avg / total       0.99      0.91      0.63      0.95      0.76      0.60     17205



In [32]:
# List the features sorted in descending order by feature importance

sorted(zip(br_forest_model.feature_importances_, X.columns), reverse=True)

[(0.08546124476285667, 'total_rec_prncp'),
 (0.07396120035144381, 'last_pymnt_amnt'),
 (0.06408288276540192, 'total_pymnt'),
 (0.05541465731823739, 'total_pymnt_inv'),
 (0.04881361258366903, 'total_rec_int'),
 (0.04004473008903708, 'issue_d'),
 (0.029415722567875235, 'int_rate'),
 (0.019875101698368678, 'mo_sin_old_rev_tl_op'),
 (0.018717659709504106, 'max_bal_bc'),
 (0.01861979450088832, 'annual_inc'),
 (0.017226536284443955, 'out_prncp'),
 (0.016825222594442725, 'dti'),
 (0.016120825265238847, 'mo_sin_old_il_acct'),
 (0.01582879708495626, 'revol_bal'),
 (0.015309368439746798, 'bc_util'),
 (0.015191853005126607, 'avg_cur_bal'),
 (0.01346432337258119, 'num_il_tl'),
 (0.013442873737888474, 'tot_hi_cred_lim'),
 (0.013406631030662988, 'tot_cur_bal'),
 (0.013349195121274613, 'out_prncp_inv'),
 (0.013177498475595415, 'total_bc_limit'),
 (0.012957404767214114, 'mths_since_rcnt_il'),
 (0.012920166981083534, 'total_rev_hi_lim'),
 (0.012697722637505371, 'installment'),
 (0.012540402140521939, '

### Easy Ensemble AdaBoost Classifier

In [27]:
# Train the EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=9)
ee_model.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=9)

In [28]:
# Calculated the balanced accuracy score
ee_predict = ee_model.predict(X_test)
print("Accuracy Score:  ", metrics.accuracy_score(y_test, ee_predict))

Accuracy Score:   0.947399011915141


In [29]:
# Display the confusion matrix
print(metrics.confusion_matrix(y_test, ee_predict))

[[16223   895]
 [   10    77]]


In [30]:
# Print the imbalanced classification report
print("False = Low Risk","\nTrue = High Risk")
print(classification_report_imbalanced(y_test, ee_predict))

False = Low Risk 
True = High Risk
                   pre       rec       spe        f1       geo       iba       sup

      False       1.00      0.95      0.89      0.97      0.92      0.84     17118
       True       0.08      0.89      0.95      0.15      0.92      0.83        87

avg / total       0.99      0.95      0.89      0.97      0.92      0.84     17205



# Results

In [38]:
print("Easy Ensemble Model: \n    ",  metrics.rand_score(y_test, ee_predict), "     Rand Socre\n    ",metrics.adjusted_rand_score(y_test, ee_predict), "   Adjusted Rand Score")
print(classification_report_imbalanced(y_test, ee_predict))

Easy Ensemble Model: 
     0.9003259584069748      Rand Socre
     0.12971483186368665    Adjusted Rand Score
                   pre       rec       spe        f1       geo       iba       sup

      False       1.00      0.95      0.89      0.97      0.92      0.84     17118
       True       0.08      0.89      0.95      0.15      0.92      0.83        87

avg / total       0.99      0.95      0.89      0.97      0.92      0.84     17205



In [39]:
print("Balanced Random Forest Model: \n    ",  metrics.rand_score(y_test, br_forest_predict), "     Rand Socre\n    ",metrics.adjusted_rand_score(y_test, br_forest_predict), "   Adjusted Rand Score")
print(classification_report_imbalanced(y_test, br_forest_predict))

Balanced Random Forest Model: 
     0.8443854794485931      Rand Socre
     0.05525779140824877    Adjusted Rand Score
                   pre       rec       spe        f1       geo       iba       sup

      False       1.00      0.92      0.63      0.96      0.76      0.60     17118
       True       0.04      0.63      0.92      0.07      0.76      0.56        87

avg / total       0.99      0.91      0.63      0.95      0.76      0.60     17205



The Easy Ensemple model has the best Adjusted Rand Score of all 6 ML Regressions. It is the best option that was tested, but I still highly recommend using a different model.